In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, "/home/amir/work/mps/src/")

using QuantumModels
using ExactDiagonalizationTools
using MatrixProductStateTools
using TensorNetAlgs
using Plots

In [ ]:
# Find the ground sate Ising Transverse Field model
lx = 100
tol = 1.e-9
maxdim = 200

mps = MPS{Float64}(lx, 2, noise=0.2);

mpo = qitf_mpo(Float64, lx, 2, 0.9)
env = initialenv(mps, mpo)

for sweep = 1:4
    println("Starting sweep number $sweep")
    dmrg2sitesweep!(mps, mpo, env, maxdim=maxdim, tol=tol, verbose=false)
    #println()
end

In [ ]:

sz, sp, sm = spinoperators(1/2)
sx = (sp + sm)/2.

data = []
half = div(lx, 2)
for l = half-2:half+2
    apply!(mps, 2 .*sx, l)
end
push!(data, [measure(mps, sz)[l] for l in 1:lx])
dt = 0.01
mpsc = convert(MPS{ComplexF64}, mps)
mpoc = convert(MPO{ComplexF64}, mpo)
env = initialenv(mpsc, mpoc)
for t=1:20
    for i = 1:50
        tdvp1sitesweep!(dt, mpsc, mpoc, env, verbose=false)
    end
    push!(data, real.([measure(mpsc, sz)[l] for l in 1:lx]))
end

In [ ]:
p = plot()
for i=1:length(data)
    plot!(p, 1:lx, -data[i])
end
p